## Titanic overlevelse

Her har vi data om passasjerer på skipet Titanic og informasjon om hvorvidt de overlevde. I denne oppgaven skal du bruke passasjerdata for å teste om det er forskjeller i overlevelsen mellom passasjergrupper og predikere om passasjeren overlevde. 

In [ ]:
# imports
import numpy as np
import pandas as pd
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, log_loss

In [ ]:
# les inn Titanic-data
titanic_df = pd.read_csv('data/titanic.csv')

Først endrer vi litt på data for å gjøre oppgaven enklere. Dette er ikke en god ide å gjøre når vi vil lage bestmulige modeller. 

In [ ]:
# del data i mål-, og prediktorvariabler
X_cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
y_col = 'Survived'
reduced_df = titanic_df.loc[:, X_cols + [y_col]].dropna()
X_df = pd.get_dummies(reduced_df.loc[:, X_cols], dtype='int')
X = X_df.values
y = reduced_df[y_col].values

Del data i trenings-, validerigns-, og testdata med størrelser 70%, 15%, 15% av data. 

In [ ]:
# dele data i trenings, validerings og testdata
# generer X_train, X_val, X_test, y_train, y_val, y_test

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.4
)

X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val, test_size=0.5
)

print(X_test)
print(y_test)

test_df = pd.DataFrame(X_test, columns=X_df.columns)
test_df[y_col] = y_test
print(test_df)

På treningsdata, test om menn og kvinner hadde forskjellige sannsynligheter for å overleve.

In [ ]:
# test om menn og kvinner hadde forskjellig overlevelse
kvinne_survival_rate = test_df[test_df["Sex_female"] == 1]["Survived"].mean()
menn_survival_rate = test_df[test_df["Sex_male"] == 1]["Survived"].mean()

print(kvinne_survival_rate)
print(menn_survival_rate)

kjonn_test = f"Prsent av menn som overlevede {round(menn_survival_rate*100,2)}% mens prosent av kvinner som overlevede {round(kvinne_survival_rate*100,2)}% "
kjonn_test

På treningsdata, test om de som overlevde hadde forskjellig mean alder enn de som ikke overlevde. 

In [ ]:
# test om de som overlevde hadde forskjellig mean alder enn de som ikke overlevde
survived = test_df[test_df["Survived"] == 1]["Age"].mean()
dead = test_df[test_df["Survived"] == 0]["Age"].mean()
alder_test = f"Døde average alder: {round(dead,2)} mens leve average alder {round(survived,2)}"
alder_test

Tren en kNN-modell med k=1, k=10 og k=50 på treningsdata. Tren også en logistisk regresjon, naive Bayes modell, tilfeldig skog og supportvektormaskin på treningsdata.

In [ ]:
# tren forskjellige modeller

model_output = {}

models = {'kNN-1': KNeighborsClassifier(n_neighbors=1),
          'kNN-10': KNeighborsClassifier(n_neighbors=10),
          'kNN-50': KNeighborsClassifier(n_neighbors=1),
          "regressor":LogisticRegression(),
          "naiveBayes":MultinomialNB(),
          "forest":RandomForestClassifier(n_estimators=100),
          "supportvector":SVC()
          }

for name, model in models.items():

    output = model.fit(X_train, y_train)

    model_output[f"{name}"] = output

print(model_output)


Sorter de ulike modellene etter nøyaktighet på valideringsdata (`sklearn.metrics.accuracy_score`).

In [ ]:
# sjekk nøyaktighet for valideringsdata
accuracies = [accuracy_score(y_val,model.predict(X_val)) for model in model_output.values()]

accuracies_df = pd.DataFrame(model_output.keys(),columns=["name"])
accuracies_df["accuracy"] = accuracies
accuracies_df.sort_values(by='accuracy', ascending=False, inplace=True)
print(accuracies_df)

I stedet for nøyaktighet er det vanlig å bruke log-loss, som tar hensyn til en probabilistisk prediksjon. Sorter de ulike modellene etter log-loss (`sklearn.metrics.log_loss`). 

In [ ]:
# sjekk log loss for valideringsdata
losses = [log_loss(y_val,model.predict(X_val)) for model in model_output.values()]
losses_df = pd.DataFrame(model_output.keys(),columns=["name"])
losses_df["loss"] = losses

losses_df.sort_values(by='loss', inplace=True)
print(losses_df)

Velg ut den beste modellen (basert på log-loss) og sjekk hvor godt den generaliserer ved å regne ut nøyaktighet og log-loss på testdata. 

In [ ]:
# sjekk generaliseringsevne
best_model = RandomForestClassifier()
best_model.fit(X_train, y_train) 
generalization_accuracy = accuracy_score(y_test,best_model.predict(X_test))
generalization_logloss = log_loss(y_test,best_model.predict(X_test))
print('Nøyaktighet:', generalization_accuracy)
print('Log-loss:', generalization_logloss)

Gi en oppsummering over hva du har gjort og hva resultatet var. 

Lest inn titanic data, delt i test/trening/validering data

Sett litt på data om hvilket kjønn som overlevde mest (kvinner)
Sett litt på gjennomsnitt alderen til døde(litt høyere) enn de som levde

Så har jeg sett på en del forskjellige modeller, trent de på train data, funnet accuracy og log_loss score. 

Da fant jeg ut at ```RandomForestClassifier``` er den beste modellen for denne dataen.

Så skulle vi se om den faktisk kan generaliseres
og drumroll . . . det kan den!

Ganske bra og, siden accruacy på test dataen er enda lavere en train dataen, log loss er en del høyere, men de andre modellene har enda høyere log loss, selv på valideringsdata.

Accuracy er litt høyere, men ikke mye. Så da har vi bevis at ```RandomForestClassifier``` er en god model hvis det skulle skje en titanic 2. (fingrene krysset at det ikke skjer)